# Introduction to Model Pipelines

In this notebook, we implement a few simple pipelines using Scikit-Learn. To illustrate the procedures, we will use the dataset on [Wine Quality available in the UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/186/wine+quality).

In [ ]:
%load_ext dotenv
%dotenv 
%run update_path.py

We use the [ucimlrepo](https://pypi.org/project/ucimlrepo/) package to easily access the data.

In [ ]:
import pandas as pd
from ucimlrepo import fetch_ucirepo 

wine_quality = fetch_ucirepo(id=186)

Inspect the object `wine_quality`, which contains useful information about the dataset. For example, use `wine_quality.keys()` to obtain the object's keys. 

In [ ]:
wine_quality.keys()

In [ ]:
wine_quality.data.keys()

In [ ]:
dt = wine_quality.data.original
X = dt.drop(columns=['quality'])
Y = 1*(dt[['quality']] >= 7).values.ravel()


In [ ]:
X.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.pairplot(X, diag_kind='hist', plot_kws={'alpha': 0.5})
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
sns.histplot(data=dt, x='quality', 
             bins=range(dt['quality'].min(), dt['quality'].max() + 2), 
             edgecolor='black', kde=False, discrete=True, multiple='stack')
plt.xlabel('Quality')
plt.ylabel('Frequency')
plt.title('Distribution of Wine Quality')
plt.show()

## Preprocessing

+ While building machine learning models, we will apply one or more [preprocessing or feature engineering steps](https://scikit-learn.org/stable/modules/preprocessing.html#) to the data.
+ The most practical way to do this is by arranging the preprocessing steps in `Pipeline` objects, which are sequential transformations applied to data. After preprocessing, we pass the data to our model. 
+ It is convenient for us to label these transformations, and there is a standard way of doing so.

## Transformations

Transformations are classes that implement `fit` and `transform` methods.

### StandardScaler

For example, transform a numerical variable by standardizing it. Standardization removes the mean of each feature and scales it by dividing each non-constant feature by its standard deviation.

$$
z = \frac{x-\mu}{\sigma}
$$


Using [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html), one can do the following:

In [ ]:
# Create a StandardScaler object
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()

# Select only numeric features for scaling
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
X_num = X[numeric_features]

# Fit the StandardScaler object with the returns data
std_scaler.fit(X_num)

In [ ]:
X_num.mean()

In [ ]:
# Transform the returns data using the fitted scaler

scaled_X = std_scaler.transform(X_num)
scaled_X_df = pd.DataFrame(scaled_X, columns=X_num.columns)

In [ ]:
scaled_X_df.mean()

In [ ]:
X_num.std()

In [ ]:
scaled_X_df.std()

###  OneHotEncoder

Categorical features can be encoded as numerical values using `OneHotEncoder`.

In [ ]:
X['color'].value_counts().plot(kind = 'bar')

Use [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) to encode a categorical variable as numerical.

Some key parameters are:

- `categories` allows you to specify the categories to work with.
- `drop`: we can drop the `'first'` value (dummy encoding) or `'if_binary'`, a convenience setting for binary values.
- `handle_unknown` allows three options, `'error'`, `'ignore'`, and `'infrequent_if_exist'`, depending on what we want to do with new values.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(drop='if_binary')
onehot.fit(X[['color']])

In [ ]:
listing_enc = onehot.transform(X[['color']])
listing_enc.toarray()

# Pipelines

+ It is impractical and costly to manipulate data "by hand". 
+ To manage data preprocessing steps within the cross-validation process, use `Pipeline` objects.
+ A [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) object allows us to sequentially apply transformation steps and, if required, a predictor.
+ `Pipeline` objects compose transforms, i.e., classes that implement `transform` and `fit` methods.
+ The purpose of `Pipeline` objects is to ensemble transforms and predictors to be used in cross-validation.
+ A `Pipeline` is defined by a list of tuples.
+ Each tuple is composed of `("name", <ColumnTransformer>)`, the name of the step, and the `<ColumnTransformer>` function of our choosing.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss, cohen_kappa_score, f1_score


In [ ]:
pipe1 = Pipeline(
    [
        ('scaler', StandardScaler()),
        ('logistic', LogisticRegression())

    ]
)
pipe1

In [ ]:

X0_train, X0_test, Y0_train, Y0_test = train_test_split(X_num, Y, test_size=0.2, random_state=42)

pipe1.fit(X0_train, Y0_train)

In [ ]:
Y_pred_train = pipe1.predict(X0_train)
Y_pred_test = pipe1.predict(X0_test)

In [ ]:
Y_poba_train = pipe1.predict_proba(X0_train)
Y_proba_test = pipe1.predict_proba(X0_test)

In [ ]:
res = {
    'accuracy_score_train': accuracy_score(Y0_train, Y_pred_train),
    'accuracy_score_test': accuracy_score(Y0_test, Y_pred_test),
    'cohen_kappa_train': cohen_kappa_score(Y0_train, Y_pred_train),
    'cohen_kappa_test': cohen_kappa_score(Y0_test, Y_pred_test),
    'log_loss_train': log_loss(Y0_train, Y_poba_train),
    'log_loss_test': log_loss(Y0_test, Y_proba_test),
    'f1_score_train': f1_score(Y0_train, Y_pred_train),
    'f1_score_test': f1_score(Y0_test, Y_pred_test)
}
res

Below, we expand the pipeline to include more variables, and further, we will work with more robust model selection pipelines.

## ColumnTransformer

+ Use [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) to apply transformers to specific columns of a DataFrame.
+ In this case, we will scale numeric variables and apply one-hot encoding to categorical columns.

In [ ]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(
    transformers=[
        ('num_transform', StandardScaler(), X_num.columns.values ),
        ('cat_transform', OneHotEncoder(handle_unknown='infrequent_if_exist', drop='if_binary'), ['color']), 
    ], remainder='drop'
)

pipe = Pipeline(
    [
        ('preproc', transformer), 
        ('logistic', LogisticRegression(l1_ratio=1.0))
    ]
)
pipe

# Model Selection

The model selection process is an iterative process in which :

+ Select schema and load data.
+ Define the pipeline and its (hyper)parameters.

    - Use ColumnTransformers to transform numeric and categorical variables.
    - Hyperparameters can be defined independently of code. 

+ Implement a splitting strategy. 

    - Use [cross_validate]() to select several metrics and operational details.

+ Measure performance.

    - [Select metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)

+ Repeat

## Training, Validation, Testing Split

+ The first splitting strategy is to use a training, validation, and test set.
+ The training set will be used to fit the model.
+ The validation set is used to evaluate hyperparameter choices.
+ Testing set is used to evaluate performance on data the model has not yet seen.
+ In this case, we want to compare two models: 

    - Decision Tree with 3 minimum samples per leaf.
    - Decision Tree with 10 minimum samples per leaf.

![](./images/03b_train_validate_test.png)

## Setting parameters in pipeline steps

+ One can obtain the parameters of a pipeline with `pipe.get_params()`.
+ We can set any parameter of a pipeline with `pipe.set_parames(**kwargs)`. 
+ The input `**kwargs` is a dictionary of the params to be modified. Params of the steps are labeled with the name of the step followed by `__` and the name of the parameter.
+ There are a few steps that we will repeat: 

    - Fit the candidate model on training data.
    - Predict on training and test data.
    - Compute training and test performance metrics.
    - Return.

+ We encapsulate this procedure in a function. 


In [ ]:
def evaluate_model(clf, X_train, Y_train, X_test, Y_test):
    clf.fit(X_train, Y_train)
    Y_pred_train = clf.predict(X_train)
    Y_pred_test = clf.predict(X_test)
    Y_proba_train = clf.predict_proba(X_train)
    Y_proba_test = clf.predict_proba(X_test)
    performance_metrics = {
        'log_loss_train': log_loss(Y_train, Y_proba_train),
        'log_loss_test': log_loss(Y_test, Y_proba_test),
        'cohen_kappa_train': cohen_kappa_score(Y_train, Y_pred_train),
        'cohen_kappa_test': cohen_kappa_score(Y_test, Y_pred_test),
        'f1_score_train': f1_score(Y_train, Y_pred_train),
        'f1_score_test': f1_score(Y_test, Y_pred_test),
        'accuracy_score_train': accuracy_score(Y_train, Y_pred_train),
        'accuracy_score_test': accuracy_score(Y_test, Y_pred_test),
    }
    return performance_metrics

In [ ]:
# Split the data
X_rest, X_test, Y_rest, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_validate, Y_train,  Y_validate = train_test_split(X_rest, Y_rest, test_size=0.2, random_state=42)


In [ ]:

# Evaluate hyperparameter configuration 2
pipe_1 = pipe.set_params(**{'logistic__C': 0.0001})
res_1 = evaluate_model(pipe_1, X_train, Y_train, X_validate, Y_validate)
res_1

In [ ]:
# Evaluate hyperparameter configuration 2
pipe_2 = pipe.set_params(**{'logistic__C': 1.0})
res_2 = evaluate_model(pipe_2, X_train, Y_train, X_validate, Y_validate)
res_2

## Cross-Validation

+ Cross-validation is a resampling method.
+ It is an iterative method applied to training data.
+ Training data is divided into folds.
+ Each fold is used once as a validation set and the rest of the folds are used for training.
+ Test data is used for final evaluation.

From [Scikit's Documentation](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-evaluating-estimator-performance), the diagram below shows the data divisions and folds used during cross-validation.

![](./images/03b_grid_search_cross_validation.png)

There are two functions that can be used for [calculating cross-validation performance scores](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-evaluating-estimator-performance): `cross_val_score()` and `cross_validate()`. The first function, [`cross_val_score()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score), is a convenience function to get quick performance calculations. We will discuss `cross_validate()`, which offers advantages over `cross_val_score()`.

## Obtaining metrics

+ Use [`cross_validate`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate) to measure one or more performance metrics and operational details.
+ There are two advantages of using this function. From [Scikit's documentation](https://scikit-learn.org/stable/modules/cross_validation.html#the-cross-validate-function-and-multiple-metric-evaluation):

>- It allows specifying multiple metrics for evaluation.
>- It returns a dict containing fit-times, score-times (and optionally training scores, fitted estimators, train-test split indices) in addition to the test score.

In [ ]:
from sklearn.model_selection import cross_validate

scoring = ['accuracy', 'f1', 'precision', 'recall', 'roc_auc', 'neg_log_loss']

dict_1 = cross_validate(pipe_1, X, Y, cv=5, scoring = scoring, return_train_score = True)

In DataFrame form:

In [ ]:
pd.DataFrame(dict_1)

In [ ]:
dict_2 = cross_validate(pipe_2, X, Y, cv=5, scoring = scoring, return_train_score = True)
pd.DataFrame(dict_2)

# About Performance

+ Notice that in order to acquire information about our model and continue development, we are spending resources: time, electricity, equipment use, etc. We are also generating data and binary objects that implement our models (e.g., fitted `Pipeline` objects).
+ For certain applications, operating performance (latency or `'score_time'`) may be as important or more important than predictive performance metrics. 
+ Every experiment throws important information, and we can log them, as well as run them systematically.

In [ ]:
pd.DataFrame(dict_1).mean()

In [ ]:
pd.DataFrame(dict_2).mean()